# Итак, чтобы построить рекомендательную систему на основе контента, необходимо:
- Для каждого продукта создать характеризующие его признаки.
- Найти показатель близости между всеми продуктами.
- Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.

# Давайте реализуем подобную рекомендательную систему на практике. 
Будем работать с датасетом, содержащим информацию об оценивании фильмов на платформе Netflix.

In [108]:
# для автозагрузки изменений в модулях без необходимости перезагрузки kelner
%load_ext autoreload
%autoreload 2

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# необходимо для корректного импорта своих модулей в JupyterLab
import sys
current_dir = sys.path[0]
project_dir = 'SF_DS_Pro'
last_position = current_dir.find(project_dir) + len(project_dir)
project_path = current_dir[:last_position]
sys.path.append(project_path)

import Handlers as hd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import set_config
set_config(transform_output = 'pandas')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# sns.set_theme('notebook')
# sns.set_palette('Set2')

plt.rcParams['figure.figsize'] = (12, 8)

In [110]:
df = pd.read_csv('data/netflix_titles.zip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


Поиск дубликатов

In [111]:
df.duplicated().sum()

0

Преобразую столбец даты

In [112]:
df['date_added'] = pd.to_datetime(df['date_added'], format='mixed')

Проверка уникальности

In [113]:
df.nunique() / df.shape[0] * 100

show_id         100.000000
type              0.025684
title           100.000000
director         51.996918
cast             87.723128
country           8.745345
date_added       19.416977
release_year      0.937460
rating            0.179787
duration          2.773854
listed_in         6.318223
description      99.768846
dtype: float64

show_id удаляю т.к. это просто номер в БД

In [114]:
df.drop(columns=['show_id'], inplace=True)

In [115]:
df.describe(include='object').T

,count,unique,top,freq
type,7787,2,Movie,5377
title,7787,7787,3%,1
director,5398,4049,"Raúl Campos, Jan Suter",18
cast,7069,6831,David Attenborough,18
country,7280,681,United States,2555
rating,7780,14,TV-MA,2863
duration,7787,216,1 Season,1608
listed_in,7787,492,Documentaries,334
description,7787,7769,Multiple women report their husbands as missin...,3


In [116]:
len_descr = df['description'].apply(lambda x: len(x))

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(stop_words='english')
df['description'] = df['description'].fillna('')
feature_matrix = model.fit_transform(df['description'])

print(f"Columns in matrix: {feature_matrix.shape[0]}")

Columns in matrix: 7787


In [118]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [119]:
# дубликатов там нет, так что операция 
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices.shape

(7787,)

In [86]:
def get_recommendations(title):
    idx = indices[title]
    # вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    # сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    # выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores = scores[1:11]
    # забираем индексы
    ind_movie = [i[0] for i in scores]
    # возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [93]:
get_recommendations("Ocean's Eleven")

4564                  Ocean's Thirteen
4                                   21
5619    Sinatra: All or Nothing at All
1347       Chocolate City: Vegas Strip
2259                      Frank & Lola
4456                              Next
1679                              Deep
4565                    Ocean's Twelve
2135    Fear and Loathing in Las Vegas
5085    Ralphie May: Imperfectly Yours
Name: title, dtype: object

In [94]:
get_recommendations("Balto")

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object